In [1]:
import numpy as np
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from torchvision import transforms
from sklearn import svm

import torch
import torch.nn as nn
import mat73
from pathlib import Path
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [46]:
# Model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(nn.Linear(2, 1))

    def forward(self, x):
        output = self.linear(x)
        # output = self.output_layer(relu3)
        return output


In [85]:
model = MLP()

In [86]:
for name, param in model.named_parameters():
    print(name)
    print(param)

linear.0.weight
Parameter containing:
tensor([[ 0.0500, -0.3470]], requires_grad=True)
linear.0.bias
Parameter containing:
tensor([0.5525], requires_grad=True)


In [ ]:
input_data = torch.tensor([[3., 2.], [1., 2.], [1., 1.]])
input_data = torch.tensor([[1., 2.]], requires_grad=True)
input_data.size()

In [113]:
input_data

tensor([[1., 2.]], requires_grad=True)

In [117]:
predict_data = model(input_data)
predict_data

tensor([[-0.0915]], grad_fn=<AddmmBackward0>)

In [93]:
weight = torch.tensor([[ 0.0500, -0.3470]])
weight.size()

torch.Size([1, 2])

In [109]:
weight @ input_data.T + 0.5525

tensor([[-0.0915]], grad_fn=<AddBackward0>)

In [115]:
input_data[0,1]

tensor(2., grad_fn=<SelectBackward0>)

In [118]:
dy_dx = torch.autograd.grad(predict_data, input_data[0,0], create_graph=True)
dy_dx

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [108]:
# d2y_dx2 = torch.autograd.grad(dy_dx, input_data, create_graph=True)
d2y_dx2 = torch.autograd.grad(dy_dx, input_data)
d2y_dx2

RuntimeError: grad can be implicitly created only for scalar outputs

In [141]:
input_data[0][0]

tensor(1., grad_fn=<SelectBackward0>)

In [126]:
t1 = 2 * (input_data ** 2)
t1

tensor([[2., 8.]], grad_fn=<MulBackward0>)

In [146]:
# torch.autograd.grad(t1, input_data, create_graph=True, grad_outputs=torch.ones_like(input_data))
torch.autograd.grad(t1, input_data[0][0], create_graph=True, grad_outputs=torch.ones_like(input_data))

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [2]:
my_dtype = 'float32'
path_practice_file = Path("D:/DL data/practice/practice_gradient.mat")
raw_input_data = mat73.loadmat(str(path_practice_file))
input_data: list
input_data = [0] * 2
input_data[0] = raw_input_data['input'][:, 0].astype(my_dtype).reshape(-1, 1)
input_data[1] = raw_input_data['input'][:, 1].astype(my_dtype).reshape(-1, 1)
input_data_tuple = tuple(input_data)
print('done')
print(input_data)

done
[array([[2.],
       [4.],
       [1.],
       [3.]], dtype=float32), array([[4.],
       [0.],
       [2.],
       [5.]], dtype=float32)]


array([[2.],
       [4.],
       [1.],
       [3.]], dtype=float32)

In [3]:
dataset_imported = torch.utils.data.DataLoader(input_data, batch_size=10)

In [ ]:
dataset_imported

In [5]:
for i, (x, t) in tqdm(enumerate(dataset_imported)):
    print(i)
    print(f'a: {x}')
    print(f'b: {t}')



1it [00:00, 249.79it/s]

0
a: tensor([[2.],
        [4.],
        [1.],
        [3.]])
b: tensor([[4.],
        [0.],
        [2.],
        [5.]])


In [6]:
x.requires_grad = True
t.requires_grad = True

In [7]:
# u = tf.pow(train_x, 2) * train_t + tf.pow(train_x, 3) * tf.pow(train_t, 2)
u = x**2 * t + x**3 * t**2

In [9]:
u.size()

torch.Size([4, 1])

In [193]:
derivs = dict()
derivs['du_dx'], derivs['du_dt'] = torch.autograd.grad(u, (x, t), create_graph=True, grad_outputs=torch.ones_like(u))
derivs['d2u_dx2'] = torch.autograd.grad(derivs['du_dx'], x, grad_outputs=torch.ones_like(derivs['du_dx']), create_graph=True)[0]
derivs['d2u_dt2'] = torch.autograd.grad(derivs['du_dt'], t, grad_outputs=torch.ones_like(derivs['du_dt']))[0]
derivs['d3u_dx2dt'] = torch.autograd.grad(derivs['d2u_dx2'], t, grad_outputs=torch.ones_like(derivs['d2u_dx2']))

In [182]:
derivs

{'du_dx': tensor([[208.],
         [  0.],
         [ 16.],
         [705.]], grad_fn=<AddBackward0>),
 'du_dt': tensor([[ 68.],
         [ 16.],
         [  5.],
         [279.]], grad_fn=<AddBackward0>),
 'd2u_dx2': (tensor([[200.],
          [  0.],
          [ 28.],
          [460.]]),),
 'd2u_dt2': (tensor([[ 16.],
          [128.],
          [  2.],
          [ 54.]]),)}

In [190]:
torch.ones_like(derivs['d2u_dx2'])

TypeError: ones_like(): argument 'input' (position 1) must be Tensor, not tuple

In [184]:
derivs['d2u_dx2']

(tensor([[200.],
         [  0.],
         [ 28.],
         [460.]], grad_fn=<AddBackward0>),)

In [185]:
derivs['du_dt']

tensor([[ 68.],
        [ 16.],
        [  5.],
        [279.]], grad_fn=<AddBackward0>)

In [2]:
my_loss_func3 = nn.BCELoss()

In [3]:
test_input = torch.rand(3, 1)
test_input

tensor([[0.9503],
        [0.0529],
        [0.2867]])

In [4]:
test_input[0]

tensor([0.9503])

In [9]:
test_output = torch.tensor([[0], [1], [1]])

In [12]:
test_output[1]

torch.Size([3, 1])

In [17]:
my_loss_func3(test_input[0], test_output[0])

tensor(3.0024)

In [32]:

my_loss_func3(test_input, test_output)


tensor(2.3971)

In [31]:
-1*(test_output[0].numpy() * np.log(test_input[0].numpy()) + (1-test_output[0].numpy())*np.log(1-test_input[0].numpy()))

array([3.0023606], dtype=float32)

In [ ]:
test_input
test_output = torch.tensor([[0], [1], [1]])
test_output.size()
test_input = test_input.to(torch.float32)
test_output = test_output.to(torch.float32)
my_loss_func3(test_input, test_output)

In [195]:
def myfun(a):
    if a == 10:
        raise Exception('over 80%')
    else:
        a = a+1
    return a


def myfun1():
    b = 1
    while b < 100:
        b = myfun(b)
        print(f'one round: {b}')


myfun1()


one round: 2
one round: 3
one round: 4
one round: 5
one round: 6
one round: 7
one round: 8
one round: 9
one round: 10


Exception: over 80%